<a href="https://colab.research.google.com/github/Shuk-Buleya/Shuk-Buleya/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langgraph langsmith langchain langchain_groq langchain_community

  Using cached langgraph-0.2.70-py3-none-any.whl.metadata (17 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled la

In [4]:
from typing import Annotated
from typing_extensions import TypedDict

In [5]:
!pip install arxiv wikipedia

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.2 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=6c45c285617953a406d6f6439b8ec008bd8171d83d63cf4bc41aa2eea24d88da
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=31c636320d627872b1092e7fe68cc203c272698266860141f926cce886542a06
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built wikipedia sgmllib3k


In [11]:
## working with tools
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arxiv_tool = ArxivQueryRun(api_wrapper = arxiv_wrapper)

wikipedia_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia_wrapper)


In [18]:
wikipedia_tool.invoke("Who is Peter Mutharika")
# arxiv_tool.invoke("digital health")

'Page: Peter Mutharika\nSummary: Arthur Peter Mutharika (born 18 July 1940) is a Malawian politician and lawyer who was President of Malawi from May 2014 to June 2020. Mutharika has worked in the field of international justice, specialising in international economic law, international law and comparat'

In [20]:
tools = [wikipedia_tool]

In [23]:
from langgraph.graph.message import add_messages
class State(TypedDict):
  messages:Annotated[list,add_messages]

In [25]:
from langgraph.graph import StateGraph, START, END

In [26]:
graph_builder = StateGraph(State)

In [27]:
from langchain_groq import ChatGroq

In [28]:
from google.colab import userdata

groq_api_key = userdata.get('firstKey')


In [35]:
llm = ChatGroq(groq_api_key=groq_api_key, model_name = "gemma2-9b-it")
llm
llm_with_tools = llm.bind_tools(tools=tools)

In [36]:
def chatbot(state:State):
  return {"messages":[llm_with_tools.invoke(state["messages"])]}

In [37]:
from langgraph.prebuilt import ToolNode, tools_condition

In [42]:
graph_builder.add_node("chatbot",chatbot)
graph_builder.add_edge(START, "chatbot")
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools",tool_node)
graph_builder.add_conditional_edges("chatbot",tools_condition)
graph_builder.add_edge("tool","chatbot")
graph_builder.add_edge("chatbot",END)


ValueError: Node `chatbot` already present.

In [41]:
graph = graph_builder.compile()

ValueError: At 'chatbot' node, 'tools_condition' branch found unknown target 'tools'